# 多向量检索器

每个文档存储多个向量通常是有益的。在许多用例中，这是有益的。 LangChain 有一个基础 MultiVectorRetriever ，这使得查询此类设置变得容易。很多复杂性在于如何为每个文档创建多个向量。本笔记本涵盖了创建这些向量和使用 MultiVectorRetriever 的一些常见方法。

为每个文档创建多个向量的方法包括：
- 较小的块：将文档分割成较小的块，然后嵌入这些块（这是 ParentDocumentRetriever）。
- 摘要：为每个文档创建摘要，将其与文档一起嵌入（或代替文档）
- 假设性问题：创建每个文档都适合回答的假设性问题，将这些问题与文档一起嵌入（或代替文档）。

请注意，这还启用了另一种添加嵌入的方法 - 手动。这很棒，因为您可以显式添加导致文档恢复的问题或查询，从而为您提供更多控制权。

In [1]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore

loaders = [
    TextLoader("/mnt/f/kk_datasets/txt/faq-4359.txt"),
    TextLoader("/mnt/f/kk_datasets/txt/faq-7923.txt"),
    TextLoader("/mnt/f/kk_datasets/txt/yjhx.md"),
    TextLoader("/mnt/f/kk_datasets/txt/agent.md"),
]

loaders

In [2]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

docs

[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-7923.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(docs)

docs


[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-7923.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准

In [4]:
# 导入本地词嵌入模型
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")


/tmp/ipykernel_34920/1980583364.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")
/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
# 用于索引子块的向量存储
vectorstore = Chroma(collection_name="full_documents", embedding_function=embeddings)


/tmp/ipykernel_34920/3936686920.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="full_documents", embedding_function=embeddings)


In [6]:
# 父文档的存储层
store = InMemoryStore()
id_key = "doc_id"

# 检索器（空启动）
retriever = MultiVectorRetriever(vectorstore=vectorstore, byte_store=store, id_key=id_key)

In [7]:
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

doc_ids


['29c8933e-468a-48ad-989c-a79873bb2f14',
 '78439bb2-fd64-4ea5-89d9-4ea817bcfdd4',
 '657814e4-424b-47a5-9142-36b6b1a59075',
 '0a1bef67-1941-40a0-a4d1-2b9db9ee44c7',
 '1ac616fa-e54c-4b5b-8d99-1b6d52670c1e',
 'c4ef2e6e-6277-43ed-a9ff-a9afbd9af2d7',
 '4bdec27e-1f3c-428c-9969-888c40bbd250']

In [8]:
# 创建较小块的分割器
from langchain_text_splitters import CharacterTextSplitter

child_text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10, length_function=len, separator="\n\n", is_separator_regex=False)

In [9]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _doc = child_text_splitter.split_documents([doc])
    for sub_doc in _doc:
        sub_doc.metadata[id_key] = _id
    sub_docs.extend(_doc)

sub_docs

Created a chunk of size 224, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 176, which is longer than the specified 100
Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 319, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 135, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 147, which is longer than the specified 100
Created a chunk of size 124, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 163, which is longer than the specified 100
Created a chunk of size 108, which is longer tha

[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '29c8933e-468a-48ad-989c-a79873bb2f14'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '29c8933e-468a-48ad-989c-a79873bb2f14'}, page_content='注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '29c8933e-468a-48ad-989c-a79873bb2f14'}, page_content='二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '29c8933e-468a-48ad-989c-a79873bb2f14'}, page_content='三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体

In [10]:
# 使用一个名为retriever的对象来向一个向量存储vectorstore添加文档
# 并且使用一个文档存储docstore来设置文档ID和文档内容的映射
# 这两个属性分别用于存储文档的向量化表示和文档的内容

retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, sub_docs)))

In [11]:
# vectorstore 单独检索小块

retriever.vectorstore.similarity_search("什么是LangGraph？")

[Document(metadata={'doc_id': '1ac616fa-e54c-4b5b-8d99-1b6d52670c1e', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 是一种基于语言模型与图谱结合的技术，旨在通过将自然语言处理（NLP）与知识图谱的优势结合，提升信息检索、语义理解和推荐系统的能力。它通过引入图谱结构，将文本数据转换为图形结构，增强了语义关联性，并能在更复杂的情境下推理和推荐。'),
 Document(metadata={'doc_id': '1ac616fa-e54c-4b5b-8d99-1b6d52670c1e', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 的核心思想是将传统的知识图谱和深度学习中的预训练语言模型（如 GPT、BERT 等）相结合。传统的知识图谱是通过节点（实体）和边（关系）构成的网络，主要用于表示实体之间的知识关系。而语言模型则能够理解并生成自然语言，在一定程度上捕捉语言的语法和语义。'),
 Document(metadata={'doc_id': '1ac616fa-e54c-4b5b-8d99-1b6d52670c1e', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 主要用于以下几个方面：\n\n- 提高知识图谱的构建效率和准确性\n- 实现智能推荐和个性化搜索\n- 增强语义搜索的精度和效率\n- 支持基于图谱的推理和决策'),
 Document(metadata={'doc_id': '1ac616fa-e54c-4b5b-8d99-1b6d52670c1e', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 通过以下两种方式将两者结合：')]

## 摘要总结

通常，摘要可能能够更准确地提炼出某个块的内容，从而实现更好的检索。在这里，我们展示如何创建摘要，然后嵌入它们。

In [12]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)

In [13]:
# 创建链

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("总结下面的文档: \n\n{doc}")
    | model
    | StrOutputParser()
)



In [14]:
docs = []

for loader in loaders:
    docs.extend(loader.load())

docs


[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-7923.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准

In [15]:
summaries = chain.batch(docs, {"max_concurrency": 5})

summaries


['总结如下：\n\n一、“0分期利息”是指消费者在购买商品时，使用花呗、招行掌上生活、工行信用卡、银联信用卡等支付方式进行分期付款时，无需支付额外的利息费用。这部分利息由华为商城承担。需要注意的是，自2023年起，华为商城将宣传中的“免息”调整为“0分期利息”，这是为了符合中国银保监会和中国人民银行的相关规定，即要求金融机构在展示分期业务时必须明确显示所有可能产生的费用和年化利率。\n\n二、目前，只有部分商品支持参与“0分期利息”活动。如果消费者同时购买多个商品，其中一些商品不支持“0分期利息”，那么整个订单将无法享受该优惠。消费者需要以支付页面上的信息为准，并关注商城未来是否会开放更多商品的“0分期利息”服务。\n\n三、若要确认订单是否成功参与“0分期利息”活动，消费者在提交订单后需要在支付方式页面选择支持“0分期利息”的分期付款选项，并选择相应的分期期数完成支付。\n\n四、如果一个订单中包含多个商品，其中部分商品支持“0分期利息”，但提交订单后发现无法享受该优惠，可能是由于这些商品不能与其他不支持的商品一起购买。在这种情况下，消费者需要取消原订单，并将支持“0分期利息”的商品和其他商品分开单独购买。此外，需要注意的是，“0分期利息”的分期期数是固定的，例如6期的“0分期利息”，并不是所有分期期数都提供该优惠。\n\n五、目前华为商城的小程序暂不支持“0分期利息”服务。',
 '华为商城众测活动总结：\n\n1. **定义与目的**：华为商城众测是一种低价试销方式，旨在通过收集用户反馈和销量数据来测试市场对新商品的接受度，并据此调整销售策略或指导商家改进产品。\n\n2. **优势**：众测商品通常提供优惠价格，但不会低于大型促销活动（如618、双十一）的价格。最终售价取决于与供应商的谈判结果。\n\n3. **适用范围**：华为商城的所有产品都可能参与众测。\n\n4. **参与方式**：\n   - 访问途径：用户可以通过在华为商城搜索“众测”来找到众测入口，并点击进入参加。\n   - 上新频次：众测频道每周一至周五不定期更新上架新产品。\n   - 活动时间：每个产品的众测时长通常为10到20天，对于热销商品可能会延长5到10天。\n\n5. **常见问题**：\n   - 商品来源：众测商品主要来自华为商城的新品和热销爆款。\n   - 价格优惠

In [17]:
# 将摘要添加到向量存储中
vectorstore = Chroma(collection_name="summaries", embedding_function=embeddings)
store = InMemoryStore()
id_key = "doc_id"
retriever = MultiVectorRetriever(vectorstore=vectorstore, byte_store=store, id_key=id_key)

doc_ids = [str(uuid.uuid4()) for _ in docs]


In [19]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]}) for i, s in enumerate(summaries)
]

summary_docs


[Document(metadata={'doc_id': '14bd613a-a552-4f8f-86c9-1455ce68d43e'}, page_content='总结如下：\n\n一、“0分期利息”是指消费者在购买商品时，使用花呗、招行掌上生活、工行信用卡、银联信用卡等支付方式进行分期付款时，无需支付额外的利息费用。这部分利息由华为商城承担。需要注意的是，自2023年起，华为商城将宣传中的“免息”调整为“0分期利息”，这是为了符合中国银保监会和中国人民银行的相关规定，即要求金融机构在展示分期业务时必须明确显示所有可能产生的费用和年化利率。\n\n二、目前，只有部分商品支持参与“0分期利息”活动。如果消费者同时购买多个商品，其中一些商品不支持“0分期利息”，那么整个订单将无法享受该优惠。消费者需要以支付页面上的信息为准，并关注商城未来是否会开放更多商品的“0分期利息”服务。\n\n三、若要确认订单是否成功参与“0分期利息”活动，消费者在提交订单后需要在支付方式页面选择支持“0分期利息”的分期付款选项，并选择相应的分期期数完成支付。\n\n四、如果一个订单中包含多个商品，其中部分商品支持“0分期利息”，但提交订单后发现无法享受该优惠，可能是由于这些商品不能与其他不支持的商品一起购买。在这种情况下，消费者需要取消原订单，并将支持“0分期利息”的商品和其他商品分开单独购买。此外，需要注意的是，“0分期利息”的分期期数是固定的，例如6期的“0分期利息”，并不是所有分期期数都提供该优惠。\n\n五、目前华为商城的小程序暂不支持“0分期利息”服务。'),
 Document(metadata={'doc_id': '37b53a04-1542-49d8-903e-cf7d204c2527'}, page_content='华为商城众测活动总结：\n\n1. **定义与目的**：华为商城众测是一种低价试销方式，旨在通过收集用户反馈和销量数据来测试市场对新商品的接受度，并据此调整销售策略或指导商家改进产品。\n\n2. **优势**：众测商品通常提供优惠价格，但不会低于大型促销活动（如618、双十一）的价格。最终售价取决于与供应商的谈判结果。\n\n3. **适用范围**：华为商城的所有产品都可能参与众测。\n\n4. **参与方式**：\n   - 访问途径：用户可以通过在

In [20]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))


In [21]:
sub_docs = retriever.vectorstore.similarity_search("众测活动是否有参与限制?")

sub_docs



[Document(metadata={'doc_id': '37b53a04-1542-49d8-903e-cf7d204c2527'}, page_content='华为商城众测活动总结：\n\n1. **定义与目的**：华为商城众测是一种低价试销方式，旨在通过收集用户反馈和销量数据来测试市场对新商品的接受度，并据此调整销售策略或指导商家改进产品。\n\n2. **优势**：众测商品通常提供优惠价格，但不会低于大型促销活动（如618、双十一）的价格。最终售价取决于与供应商的谈判结果。\n\n3. **适用范围**：华为商城的所有产品都可能参与众测。\n\n4. **参与方式**：\n   - 访问途径：用户可以通过在华为商城搜索“众测”来找到众测入口，并点击进入参加。\n   - 上新频次：众测频道每周一至周五不定期更新上架新产品。\n   - 活动时间：每个产品的众测时长通常为10到20天，对于热销商品可能会延长5到10天。\n\n5. **常见问题**：\n   - 商品来源：众测商品主要来自华为商城的新品和热销爆款。\n   - 价格优惠：众测商品的价格通常与618或双十一促销价持平，是该时期内的最低售价。\n   - 质量保证：众测商品为量产的正品新品，质量与正式上架商品相同。\n   - 发货时间：发货时间请以商品页面显示为准。\n   - 支付时限：提交订单后最长付款时间为24小时，逾期未支付将自动取消订单。\n   - 订单查询：成功下单后，用户可以通过华为商城APP、WAP版或电脑网页版登录账户在“我的订单”中查询。\n   - 取消订单：发货前消费者可以随时取消订单。\n   - 退款时效：取消订单后的退款将在3到5个工作日内完成。\n   - 参与次数限制：用户账号无众测活动参与次数限制，也没有限购政策，但每次只能购买一个商品。如需购买多个，需要多次参加众测活动。'),
 Document(metadata={'doc_id': '14bd613a-a552-4f8f-86c9-1455ce68d43e'}, page_content='总结如下：\n\n一、“0分期利息”是指消费者在购买商品时，使用花呗、招行掌上生活、工行信用卡、银联信用卡等支付方式进行分期付款时，无需支付额外的利息费用。这部分利息由华为商城承担。需要注意的是，自2023年